# Time to slice & dice

## Load dataset

In [1]:
from datasets import load_dataset

In [2]:
squad = load_dataset("squad", split="train")

In [3]:
squad[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

## Shuffle

In [4]:
squad_shuffled = squad.shuffle(seed=666)
squad_shuffled[0]

{'id': '5727cc873acd2414000deca9',
 'title': 'Oklahoma',
 'context': 'Oklahoma is the 20th largest state in the United States, covering an area of 69,898 square miles (181,035 km2), with 68,667 square miles (177847 km2) of land and 1,281 square miles (3,188 km2) of water. It is one of six states on the Frontier Strip and lies partly in the Great Plains near the geographical center of the 48 contiguous states. It is bounded on the east by Arkansas and Missouri, on the north by Kansas, on the northwest by Colorado, on the far west by New Mexico, and on the south and near-west by Texas.',
 'question': 'Where does Oklahoma rank by land area?',
 'answers': {'text': ['20th'], 'answer_start': [16]}}

In [5]:
dataset = squad.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 78839
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 8760
    })
})

## Select

In [6]:
indices = [0, 10, 20, 40, 80]
examples = squad.select(indices)
examples

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5
})

In [7]:
sample = squad.shuffle().select(range(5)); sample

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5
})

## Filter

In [8]:
squad_filtered = squad.filter(lambda x: x["title"].startswith("L"))
squad_filtered

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 2049
})

## Rename

In [9]:
squad.rename_column("context", "passages")

Dataset({
    features: ['id', 'title', 'passages', 'question', 'answers'],
    num_rows: 87599
})

## Remove

In [10]:
squad.remove_columns(["id", "title"])

Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 87599
})

## Flatten

In [11]:
squad.flatten()

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers.text', 'answers.answer_start'],
    num_rows: 87599
})

## Map

In [12]:
def lowercase_title(example): return {'title': example["title"].lower()}

In [13]:
squad_lowercase = squad.map(lowercase_title)

In [14]:
squad_lowercase.shuffle(seed=42)["title"][:5]

['egypt',
 'ann_arbor,_michigan',
 'rule_of_law',
 'samurai',
 'group_(mathematics)']

## Batched Map

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_title(example): return tokenizer(example["title"])

squad.map(tokenize_title, batched=True, batch_size=500)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask'],
    num_rows: 87599
})

## Drug Review Dataset

In [16]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip

--2023-09-17 19:53:18--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip.3’

drugsCom_raw.zip.3      [              <=>   ]  41.00M  14.6MB/s    in 2.8s    

2023-09-17 19:53:21 (14.6 MB/s) - ‘drugsCom_raw.zip.3’ saved [42989872]

Archive:  drugsCom_raw.zip
replace drugsComTest_raw.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [17]:
data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}

In [18]:
drug_dataset = load_dataset("csv", data_files=data_files, delimiter = "\t")

In [19]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [20]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
drug_sample[:3]

{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

In [21]:
for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

In [22]:
drug_dataset = drug_dataset.rename_column(
    original_column_name = "Unnamed: 0", new_column_name="patient_id"
)

In [23]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [24]:
for split in drug_dataset.keys():
    print(split)
    print("Number of drugs ", len(drug_dataset[split].unique("drugName")))
    print("Number of conditions ", len(drug_dataset[split].unique("condition")))

train
Number of drugs  3436
Number of conditions  885
test
Number of drugs  2637
Number of conditions  709


In [25]:
def lowercase_condition(example):
    return {"condition": example["condition"].lower()}


In [26]:
def filter_nones(x):
    return x["condition"] is not None

In [27]:
drug_dataset.filter(filter_nones)

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 160398
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53471
    })
})

In [28]:
(lambda x: x * x)(3)

9

In [29]:
(lambda base, height: 0.5 * height * base)(4, 8)

16.0

In [30]:
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

In [31]:
drug_dataset = drug_dataset.map(lowercase_condition)

In [32]:
drug_dataset["train"]["condition"][:3]

['left ventricular dysfunction', 'adhd', 'birth control']

In [33]:
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}

In [34]:
drug_dataset = drug_dataset.map(compute_review_length)

In [35]:
drug_dataset["train"][0]

{'patient_id': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 17}

In [36]:
drug_dataset["train"].sort("review_length")[:3]

{'patient_id': [111469, 13653, 53602],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse'],
 'condition': ['hepatitis c', 'adhd', 'birth control'],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'rating': [10.0, 10.0, 10.0],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'usefulCount': [41, 3, 0],
 'review_length': [1, 1, 1]}

In [37]:
drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30)

In [38]:
drug_dataset.num_rows

{'train': 138514, 'test': 46108}

In [39]:
sorted_drug_dataset = drug_dataset['test'].sort('review_length', reverse=True)

In [40]:
sorted_drug_dataset[:1]

{'patient_id': [45000],
 'drugName': ['Fluoxetine'],
 'condition': ['obsessive compulsive disorde'],
 'review': ['"I don&rsquo;t find a lot of positive stories about antidepressants, or I find stories where people are taking the antidepressant the wrong way.\r\n\r\nI wanted to share my experience.  A positive one.\r\n\r\nI&rsquo;ve had generalized anxiety disorder, SEVERE OCD, and panic disorder for as long as I can remember.  My first memory of having an episode was when I was 4 years old at my kindergarten interview.  I feel as though I was born with the illnesses mentioned above, right from the womb.  When I was a child I was extremely anxious, had bad separation anxiety from my parents and had extreme OCD, I was just a kid and thought that the way I was feeling is how all kids felt, I didn&rsquo;t realize that I was different.  This went on, and got even worse in middle school.  I began developing trichtilomania in middle school.  In high school I went from being a 90% above studen

In [41]:
import html

text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

In [42]:
drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x["review"])})

In [43]:
drug_dataset['test'][:1]

{'patient_id': [163740],
 'drugName': ['Mirtazapine'],
 'condition': ['depression'],
 'review': ['"I\'ve tried a few antidepressants over the years (citalopram, fluoxetine, amitriptyline), but none of those helped with my depression, insomnia & anxiety. My doctor suggested and changed me onto 45mg mirtazapine and this medicine has saved my life. Thankfully I have had no side effects especially the most common - weight gain, I\'ve actually lost alot of weight. I still have suicidal thoughts but mirtazapine has saved me."'],
 'rating': [10.0],
 'date': ['February 28, 2012'],
 'usefulCount': [22],
 'review_length': [68]}

In [44]:
new_drug_dataset = drug_dataset.map(
    lambda x: {"review": [html.unescape(o) for o in x["review"]]}, batched=True
)

In [45]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True)

In [46]:
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)

CPU times: user 39.1 ms, sys: 38.9 ms, total: 78 ms
Wall time: 146 ms


In [47]:
slow_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)

In [48]:
def slow_tokenize_function(examples):
    return slow_tokenizer(examples["review"], truncation=True)

In [49]:
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

CPU times: user 2min 18s, sys: 1.77 s, total: 2min 20s
Wall time: 2min 25s


In [50]:
%time tokenized_dataset = drug_dataset.map(slow_tokenize_function, batched=True)

CPU times: user 913 ms, sys: 60.3 ms, total: 973 ms
Wall time: 1.15 s


In [51]:
%time tokenized_dataset = drug_dataset.map(slow_tokenize_function, batched=False)

CPU times: user 827 ms, sys: 51 ms, total: 878 ms
Wall time: 971 ms


In [52]:
def tokenize_and_split(examples):
    return tokenizer(
        examples["review"],
        truncation=True, 
        max_length=128,
        return_overflowing_tokens=True)

In [53]:
result = tokenize_and_split(drug_dataset["train"][0])

In [54]:
result

{'input_ids': [[101, 107, 1422, 1488, 1110, 9079, 1194, 1117, 2223, 1989, 1104, 1130, 19972, 11083, 119, 1284, 1245, 4264, 1165, 1119, 1310, 1142, 1314, 1989, 117, 1165, 1119, 1408, 1781, 1103, 2439, 13753, 1119, 1209, 1129, 1113, 119, 1370, 1160, 1552, 117, 1119, 1180, 6374, 1243, 1149, 1104, 1908, 117, 1108, 1304, 172, 14687, 1183, 117, 1105, 7362, 1111, 2212, 129, 2005, 1113, 170, 2797, 1313, 1121, 1278, 12020, 113, 1304, 5283, 1111, 1140, 119, 114, 146, 1270, 1117, 3995, 1113, 6356, 2106, 1105, 1131, 1163, 1106, 6166, 1122, 1149, 170, 1374, 1552, 119, 3969, 1293, 1119, 1225, 1120, 1278, 117, 1105, 1114, 2033, 1146, 1107, 1103, 2106, 119, 1109, 1314, 1160, 1552, 1138, 1151, 2463, 1714, 119, 1124, 1110, 150, 21986, 3048, 1167, 5340, 1895, 1190, 1518, 102], [101, 119, 1124, 1110, 1750, 6438, 113, 170, 1363, 1645, 114, 117, 1750, 172, 14687, 1183, 119, 1124, 1110, 11566, 1155, 1103, 1614, 1119, 1431, 119, 8007, 1117, 4658, 1110, 1618, 119, 1284, 1138, 1793, 1242, 1472, 23897, 1105, 117

In [55]:
len(result['input_ids'])

2

In [56]:
len(result['input_ids'][0]), len(result['input_ids'][1]) 

(128, 49)

In [57]:
tokenized_dataset = drug_dataset.map(tokenize_and_split)

In [58]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 138514
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 46108
    })
})

In [59]:
len(tokenized_dataset['train']['input_ids'][7])

2

In [60]:
tokenized_dataset_removed = drug_dataset.map(tokenize_and_split, batched=True, remove_columns=drug_dataset["train"].column_names)

In [61]:
tokenized_dataset_removed

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 206772
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 68876
    })
})

In [62]:
len(tokenized_dataset_removed['train']['input_ids'][0]), len(tokenized_dataset_removed['train']['input_ids'][1]), len(tokenized_dataset_removed['train']['input_ids'][2])

(128, 49, 128)

In [63]:
def tokenize_and_split(examples):
    result = tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True)

    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

In [64]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)

In [65]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 206772
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 68876
    })
})

In [66]:
len(tokenized_dataset['train']['input_ids'][0]), len(tokenized_dataset['train']['input_ids'][1]), len(tokenized_dataset['train']['input_ids'][2])

(128, 49, 128)

## Datasets + Dataframes = love

In [67]:
drug_dataset.set_format("pandas")

In [68]:
drug_dataset["train"][:3]

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89


In [69]:
train_df = drug_dataset["train"][:]

In [70]:
frequencies = train_df["condition"].value_counts().to_frame().reset_index().rename(columns={"index": "condition", "condition": "frequency"})

In [71]:
frequencies.head()

,frequency,count
0,birth control,27655
1,depression,8023
2,acne,5209
3,anxiety,4991
4,pain,4744


In [72]:
from datasets import Dataset

freq_dataset = Dataset.from_pandas(frequencies)
freq_dataset

Dataset({
    features: ['frequency', 'count'],
    num_rows: 819
})

lets compute average rating per drug

In [73]:
df_drug_average_rating = train_df.groupby('drugName')['rating'].mean().to_frame().reset_index()

In [74]:
df_drug_average_rating.head()

,drugName,rating
0,A + D Cracked Skin Relief,10.000000
1,A / B Otic,10.000000
2,Abacavir / dolutegravir / lamivudine,7.953488
3,Abacavir / lamivudine / zidovudine,9.000000
4,Abatacept,7.312500


In [75]:
drug_average_rating = Dataset.from_pandas(df_drug_average_rating)

In [76]:
drug_average_rating

Dataset({
    features: ['drugName', 'rating'],
    num_rows: 3052
})

In [77]:
drug_dataset.reset_format()

## Create validation set

In [78]:
drug_dataset_clean = drug_dataset["train"].train_test_split(train_size=0.8, seed=42)

In [79]:
drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")

In [80]:
drug_dataset_clean["test"] = drug_dataset["test"]

In [81]:
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

## Saving a dataset

In [82]:
drug_dataset_clean.save_to_disk("drug-reviews")

Saving the dataset (0/1 shards):   0%|          | 0/110811 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27703 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/46108 [00:00<?, ? examples/s]

In [83]:
!ls drug-reviews

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
dataset_dict.json test              train             validation


In [86]:
from datasets import load_from_disk

In [87]:
drug_dataset_reloaded = load_from_disk("drug-reviews")

In [88]:
drug_dataset_reloaded

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

In [90]:
for split, dataset in drug_dataset_clean.items():
    dataset.to_json(f"drug-reviews-{split}.json1")

Creating json from Arrow format:   0%|          | 0/111 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/47 [00:00<?, ?ba/s]

In [91]:
!head -n 1 drug-reviews-train.json1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{"patient_id":89879,"drugName":"Cyclosporine","condition":"keratoconjunctivitis sicca","review":"\"I have used Restasis for about a year now and have seen almost no progress.  For most of my life I've had red and bothersome eyes. After trying various eye drops, my doctor recommended Restasis.  He said it typically takes 3 to 6 months for it to really kick in but it never did kick in.  When I put the drops in it burns my eyes for the first 30 - 40 minutes.  I've talked with my doctor about this and he said it is normal but should go away after some time, but it hasn't. Every year around spring time my eyes get terrible irritated  and this year has been the same (maybe 

In [92]:
data_files = {
    "train": "drug-reviews-train.json1",
    "validation": "drug-reviews-validation.json1",
    "test": "drug-reviews-test.json1"
}

drug_dataset_reloaded = load_dataset("json", data_files = data_files)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [93]:
drug_dataset_reloaded

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})